In [1]:
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from utility.others import clear_all
import time
from functions import DataGenerator,PredictGenerator,create_model,dot_distance,exp_loss,\
                        sigmoid_dot_distance,dot_sigmoid_distance,cross_entropy_loss,l2_distance,\
                        margin_loss_fun_factory,Xception_reduced,dot_distance_neg,FineTune
from utility.grayscale_imagenet import Xception_greyscale
from utility.grayscale_DenseNet import DenseNet_greyscale
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
#from tensorflow.keras.applications import DenseNet121

In [2]:
# constant that unlikely to change
color = False
shapes = (224,224,1)
HalfBatch = 4

### set up generator

In [3]:
with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
    Ids_train = pickle.load(f)
with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
    newWhale_train = pickle.load(f)
with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
    Ids_val = pickle.load(f)
with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
    newWhale_val = pickle.load(f)

In [4]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

gen_train = DataGenerator(Ids_train,newWhale_train,transform,color=color,HalfBatch=HalfBatch)
gen_val = DataGenerator(Ids_val,newWhale_val,transform_test,color=color,HalfBatch=HalfBatch)

### build model

In [5]:
conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
#conv_base = Xception_greyscale((256,256,1),'max',False)
# conv_base = Xception_reduced((256,256,1))

In [6]:
train_model,feature_model = create_model(1e-3,dot_distance_neg,cross_entropy_loss,\
                                         conv_base,color,[512],[None])

### training

In [18]:
# conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
# train_model,feature_model = create_model(1e-3,dot_distance_neg,cross_entropy_loss,\
#                                          conv_base,color,[512],[None])
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=40,
          use_multiprocessing=True,workers=2,max_queue_size=20,
          callbacks=[FineTune(['conv5','conv4','conv3','conv2'],
                              cross_entropy_loss,
                              tf.keras.optimizers.Adam,
                              save_model=['Models/DenseNet_cross_entropy_fine_tune.h5',\
                                          'Models/DenseNet_feature_cross_entropy_fine_tune.h5'])])
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/40
1667/1667 [==============================] - 110s 66ms/step - loss: 0.5000 - val_loss: 3.6906
Epoch 2/40
1667/1667 [==============================] - 101s 60ms/step - loss: 0.5018 - val_loss: 3.2417
Epoch 3/40
1667/1667 [==============================] - 101s 61ms/step - loss: 0.5264 - val_loss: 4.5342
Epoch 4/40
1667/1667 [==============================] - 101s 61ms/step - loss: 0.5291 - val_loss: 4.8114
Epoch 5/40
1667/1667 [==============================] - 101s 60ms/step - loss: 0.5459 - val_loss: 4.7852
Epoch 6/40
1667/1667 [==============================] - 108s 64ms/step - loss: 0.5460 - val_loss: 5.1331
Epoch 7/40
1667/1667 [==============================] - 121s 73ms/step - loss: 0.5543 - val_loss: 4.8775
Epoch 8/40
1667/1667 [==============================] - 175s 105ms/step - loss: 0.3994 - val_loss: 4.1507
Epoch 9/40
1667/1667 [==============================] - 152s 91ms/step - loss: 0.3573 - val_loss: 3.2967
Epoch 10/40
1667/1667 [==============================]

In [20]:
11333.168945789337/60/60

3.1481024849414827

In [9]:
# conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
# train_model,feature_model = create_model(1e-3,dot_distance_neg,cross_entropy_loss,\
#                                          conv_base,color,[512],[None])
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
713/713 [==============================] - 72s 102ms/step - loss: 0.5547 - val_loss: 2.4872
Epoch 2/5
713/713 [==============================] - 63s 88ms/step - loss: 0.5334 - val_loss: 2.4161
Epoch 3/5
713/713 [==============================] - 84s 118ms/step - loss: 0.5455 - val_loss: 2.4620
Epoch 4/5
713/713 [==============================] - 126s 176ms/step - loss: 0.5531 - val_loss: 2.2825
Epoch 5/5
713/713 [==============================] - 125s 175ms/step - loss: 0.5538 - val_loss: 2.8859
time:470.37463998794556


Fine-Tune

In [10]:
feature_model.layers[0].trainable=True

train = False
for layer in feature_model.layers[0].layers:
    if 'conv5' in layer.name:
        train = True
    layer.trainable = train

train_model.compile(loss=cross_entropy_loss,optimizer=tf.keras.optimizers.Adam(lr=5e-4))

In [13]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
713/713 [==============================] - 156s 219ms/step - loss: 0.5210 - val_loss: 2.0644
Epoch 2/5
713/713 [==============================] - 141s 198ms/step - loss: 0.4816 - val_loss: 1.7568
Epoch 3/5
713/713 [==============================] - 141s 198ms/step - loss: 0.4722 - val_loss: 1.8767
Epoch 4/5
713/713 [==============================] - 141s 198ms/step - loss: 0.4557 - val_loss: 1.9497
Epoch 5/5
713/713 [==============================] - 141s 198ms/step - loss: 0.4555 - val_loss: 1.4881
time:725.0472626686096


In [14]:
train_model.save('Models/DenseNet_greyscale_cross_entropy.h5')
feature_model.save('Models/feature_model_DenseNet_greyscale_cross_entropy.h5')

In [7]:
# conv_base = Xception_reduced((256,256,1))
# train_model,feature_model = create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
#                                         conv_base,color,[512,512],['relu',None])
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=10,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/10
713/713 [==============================] - 248s 348ms/step - loss: -0.4861 - val_loss: -0.6561
Epoch 2/10
713/713 [==============================] - 246s 345ms/step - loss: -0.6663 - val_loss: -0.9618
Epoch 3/10
713/713 [==============================] - 246s 345ms/step - loss: -0.7361 - val_loss: -1.0430
Epoch 4/10
713/713 [==============================] - 261s 366ms/step - loss: -0.8041 - val_loss: -1.0988
Epoch 5/10
713/713 [==============================] - 270s 378ms/step - loss: -0.8884 - val_loss: -1.0215
Epoch 6/10
713/713 [==============================] - 272s 382ms/step - loss: -0.9159 - val_loss: -1.1343
Epoch 7/10
713/713 [==============================] - 368s 516ms/step - loss: -0.9585 - val_loss: -1.1587
Epoch 8/10
713/713 [==============================] - 369s 518ms/step - loss: -0.9702 - val_loss: -1.1007
Epoch 9/10
713/713 [==============================] - 371s 520ms/step - loss: -1.0204 - val_loss: -1.2200
Epoch 10/10
713/713 [=========================

In [9]:
train_model.save('Models/train_model_create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
                                         Xception_reduced((256,256,1)),color,[512,512],[relu,None]).h5')
feature_model.save('Models/feature_model_create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
                                         Xception_reduced((256,256,1)),color,[512,512],[relu,None]).h5')

In [17]:
# create_model(5e-4,dot_distance,exp_loss,Xception_greyscale((256,256,1),'max',False),color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 165s 232ms/step - loss: 0.9346 - val_loss: 3.0979
Epoch 2/5
714/714 [==============================] - 165s 231ms/step - loss: 0.9273 - val_loss: 2.7641
Epoch 3/5
714/714 [==============================] - 165s 231ms/step - loss: 0.9310 - val_loss: 2.9462
Epoch 4/5
714/714 [==============================] - 165s 231ms/step - loss: 0.9089 - val_loss: 5.3465
Epoch 5/5
714/714 [==============================] - 165s 231ms/step - loss: 0.9125 - val_loss: 4.3415
time:824.5533976554871


In [39]:
# create_model(5e-2,dot_sigmoid_distance,exp_loss,Xception_greyscale((256,256,1),'max',False),color)
# this just push everything to huge negative number. So after sigmoid all zero

In [5]:
# create_model(5e-4,sigmoid_dot_distance,exp_loss,Xception_greyscale((256,256,1),'max',False),color)
# loss does not decrease

In [8]:
# train_model,feature_model = create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
#                                         conv_base,color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 89s 125ms/step - loss: -0.5730 - val_loss: -0.6638
Epoch 2/5
714/714 [==============================] - 85s 119ms/step - loss: -0.6436 - val_loss: -0.6496
Epoch 3/5
714/714 [==============================] - 132s 184ms/step - loss: -0.6517 - val_loss: -0.6050
Epoch 4/5
714/714 [==============================] - 164s 230ms/step - loss: -0.6505 - val_loss: -0.6908
Epoch 5/5
714/714 [==============================] - 164s 230ms/step - loss: -0.6815 - val_loss: -0.6497
time:634.2249140739441


Fine-Tune

In [12]:
feature_model.layers[0].trainable=True

In [14]:
for layer in feature_model.layers[0].layers:
    if layer.name in ['block14_sepconv2','block14_sepconv1']:
        layer.trainable = True
    else:
        layer.trainable = False

In [19]:
train_model.compile(loss=margin_loss_fun_factory(0.1,5),optimizer=tf.keras.optimizers.Adam(lr=1e-4))

In [20]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 180s 252ms/step - loss: -0.8127 - val_loss: -0.7030
Epoch 2/5
714/714 [==============================] - 178s 250ms/step - loss: -0.9215 - val_loss: -0.7280
Epoch 3/5
714/714 [==============================] - 179s 250ms/step - loss: -1.0445 - val_loss: -0.6875
Epoch 4/5
714/714 [==============================] - 179s 251ms/step - loss: -1.0952 - val_loss: -0.5705
Epoch 5/5
714/714 [==============================] - 179s 250ms/step - loss: -1.0889 - val_loss: -0.6476
time:895.5712113380432


In [21]:
train_model.save('Models/train_l2_distance_margin_loss_fun_factory(0.1,5).h5')
feature_model.save('Models/feature_l2_distance_margin_loss_fun_factory(0.1,5).h5')